In [4]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Load pre-trained BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fine-tuning steps would go here (omitted for brevity)

# Function to answer questions
def answer_question(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

# Example usage
context = "Your specific text goes here."
question = "What question do you want to ask?"

answer = answer_question(question, context)
print(f"Question: {question}")
print(f"Answer: {answer}")

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Arin Zingade\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForQuestionAnswering, BertConfig

# Step 1: Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Step 2: Prepare the data (simple example with one data point)
data = [
    {
        "context": "TensorFlow is a free and open-source software library for machine learning. It was developed by Google.",
        "question": "Who developed TensorFlow?",
        "answer": "Google",
        "start": 81,  # Start character position of the answer in the context
        "end": 87     # End character position of the answer in the context
    }
]

# Function to encode the data
def encode_data(tokenizer, data, max_length=512):
    inputs = []
    start_positions = []
    end_positions = []
    
    for item in data:
        encoded = tokenizer.encode_plus(
            item['question'],
            item['context'],
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_offsets_mapping=True
        )
        # Find tokens that were created for the answer
        offsets = encoded['offset_mapping']
        answer = item['answer']
        start_char = item['start']
        end_char = item['end']

        # Convert character positions to token positions
        start_token = None
        end_token = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char and end >= start_char:
                start_token = idx
            if start <= end_char and end >= end_char:
                end_token = idx
                break

        inputs.append(encoded['input_ids'])
        start_positions.append(start_token)
        end_positions.append(end_token)

    # Create TensorFlow datasets
    dataset = tf.data.Dataset.from_tensor_slices({
        'input_ids': inputs,
        'token_type_ids': [x['token_type_ids'] for x in inputs],
        'attention_mask': [x['attention_mask'] for x in inputs],
        'start_positions': start_positions,
        'end_positions': end_positions
    })

    return dataset

# Step 3: Encode the data
encoded_dataset = encode_data(tokenizer, data)

# Step 4: Define the training setup
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss)

# Step 5: Train the model
model.fit(encoded_dataset.batch(1), epochs=3)

# Function to ask a question
def ask_question(model, tokenizer, question, context):
    input_dict = tokenizer.encode_plus(question, context, return_tensors='tf')
    outputs = model(input_dict)
    start_logits, end_logits = outputs.start_logits, outputs.end_logits
    start_index = tf.argmax(start_logits, axis=-1).numpy()[0]
    end_index = tf.argmax(end_logits, axis=-1).numpy()[0] + 1
    answer_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"][0, start_index:end_index])
    return tokenizer.convert_tokens_to_string(answer_tokens)

# Step 6: Use the model to answer a question
context = "TensorFlow is a free and open-source software library for machine learning. It was developed by Google."
question = "What is TensorFlow?"
answer = ask_question(model, tokenizer, question, context)
print(f"Answer: {answer}")


c:\Users\Arin Zingade\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Arin Zingade\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.